# Extracción, transformación y carga de los datos

### Librerias útiles

Una vez que se cuenta con el conjunto de datos etiquetado, éste debe ser dividido en tres subconjuntos que llamaremos "entrenamiento", "validación" y "prueba", divididos en 70%, 10% y 20% de los datos. Esta división es común dentro de la ciencia de datos y está basada en el *principio de Pareto*.

Además de la división es necesario transformar el texto de cada tweet, pues recordemos cada texto puede contener emoticonos 🇧🇷🇨🇷🇳🇮🇲🇽, hashtags *#VivaLaMigración*, menciones a usuarios [@el_BID](https://twitter.com/el_bid?lang=es) y links a páginas externas. Algunos de estos elementos pueden contener información útil para la detección de sentimiento y/o xenofobia (como los hashtags), pero otros pueden no ser nada informativos (como los nombres de usuario).

##### Scikit learn
Se trata de una librería de código abierto enfocada en proveer herramientas de aprendizaje de máquinas tales como modelos estadísticos y matemáticos, así como métricas de evaluación comunes en algoritmos de aprendizaje de máquinas.
<figure>
    <img src="./assets/images/scikit.png"
         alt="scikit-learn logo"
         style="max-width: 20%; height: auto">
    <figcaption>scikit-learn logo</figcaption>
</figure>

Esta librería nos permitirá dividir el conjunto de datos en los subconjuntos mencionados.

In [1]:
#Imports

#data reading
import pandas as pd

#data processing
from pysentimiento.preprocessing import preprocess_tweet
import re

#scikit-learn creacion de conjuntos de entrenamiento, prueba y validación
from sklearn.model_selection import train_test_split

### Cargando los datos etiquetados

In [2]:
data = pd.read_csv('./assets/data/xeno_data_workshop.csv')
data.sample(n=5)

,id,text,date,label
9807,1254872614713114624,Gran trabajo de coordinación y apoyo humanitar...,2021-10-06,0
6246,1258399516371361792,@Oscar08464952 @Arizasoy Hola sabes de alguna ...,2021-09-10,0
7158,1360251914777157632,@rdco15 @MigracionCol Si. Pero de manera regul...,2021-10-16,0
7220,1182294468738854912,Como pueden los gobiernos exigir requisitos de...,2021-11-13,0
83,1138155946428043264,Implementarán carnet fronterizo como control m...,2021-11-20,0


In [3]:
print('Tweet marcado como xenofóbo:\n {}\nTweet marcado como NO xenofóbo:\n {}'.format(data[data.label==1].sample(n=1).text.values[0], data[data.label==0].sample(n=1).text.values[0]))

Tweet marcado como xenofóbo:
 Una imagen dicen mil Palabras".  Si ven la cara de Cojudo? Como que no quebrara un Plato".... Aca en ECUADOR. Nos sirve Como un ejemplo?  LA INVASION DE EXTRANJEROS MAS VENEZOLANOS" Son Asi.  Carita de Dios. Pero Recuerda Es mejor no conocerlos?  De lejitos"
Tweet marcado como NO xenofóbo:
 #Nacional | Alrededor de 3⃣0⃣ migrantes centroamericanos fueron asegurados en un operativo realizado por elementos de la Guardia Nacional. 🚃   https://t.co/11exZf9vxG


### Limpieza de textos

In [4]:
example_text = 'Que opinas de la reformas Migratorias?👀👀 mi buen amigo @Pepe?\nYo pienso que DEbieron facilitar la entrada de los inmigrantes 💁🇧🇷🇨🇷🇳🇮🇲🇽💪#YoTeApoyo eeeexxxxxx jajajajaja.'
bid_url = ' Página oficial del BID:          https://www.iadb.org/es '
example_text = example_text + bid_url
print(example_text)

Que opinas de la reformas Migratorias?👀👀 mi buen amigo @Pepe?
Yo pienso que DEbieron facilitar la entrada de los inmigrantes 💁🇧🇷🇨🇷🇳🇮🇲🇽💪#YoTeApoyo eeeexxxxxx jajajajaja. Página oficial del BID:          https://www.iadb.org/es 


In [5]:
preprocess_tweet(example_text, user_token='@usuario', url_token='url', preprocess_hashtags=True, demoji=True, shorten=2, normalize_laughter=True, hashtag_token='hashtag')

'Que opinas de la reformas Migratorias? emoji ojos emoji  emoji ojos emoji  mi buen amigo @usuario?\nYo pienso que DEbieron facilitar la entrada de los inmigrantes  emoji persona de mostrador de información emoji  emoji bandera brasil emoji  emoji bandera costa rica emoji  emoji bandera nicaragua emoji  emoji bandera méxico emoji  emoji bíceps flexionado emoji hashtag yo te apoyo eexx jaja. Página oficial del BID:  url'

Podemos observar que el texto cambia los links por la palabra definida "url", los nombres de usuario por "@usuario", separa las palabras contenidas en un hashtag y además agrega "hashtag", normaliza las expresiones de risa y las letras repetidas. Describe el contenido de cada emoticono.

Las palabras definidas serán muy útiles para el modelo, pues éste puede ser configurado para que considere dichas palabras como elementos estructurales y no como palabras.

Sin embargo, se observa que el texto contiene espacios, mantiene los acentos, los saltos de línea y las mayúsculas. Estas características podrían ser no deseadas según el objetivo que se tiene y el modelo que se empleará para clasificar los textos. Ya que existen modelos que hacen diferencia entre estas características y otros que no.

In [6]:
#remove extra spaces
re.compile(r'\s+').sub('#', bid_url)

'#Página#oficial#del#BID:#https://www.iadb.org/es#'

In [7]:
#juntemos estos conceptos para crear una función que nos permita limpiar un tweet
def clean_tweet(tweet, user_token='@usuario', url_token='url', hashtag_token='hashtag', preprocess_hashtags=True, demoji=True, shorten=2, normalize_laughter=True):
    tweet = str(tweet)
    tweet = preprocess_tweet(tweet, user_token=user_token, url_token=url_token, preprocess_hashtags=preprocess_hashtags, demoji=demoji, shorten=shorten, normalize_laughter=normalize_laughter, hashtag_token=hashtag_token)
    tweet = re.compile(r'\s+').sub(' ', tweet)
    return tweet

data['text'] = data.text.apply(lambda x: clean_tweet(x))
#remove initial and final spaces
data['text'] = data.text.str.strip()

In [8]:
data.sample(n=1).values

array([[1342852606264291328,
        '@usuario Jajaj dales alojamiento tu po qué eres defensor de la inmigración masiva e ilegal que totalmente irrespetuosa con nuestro país por como lo tratan y lo que hacen en el',
        '2021-07-04', 1]], dtype=object)

### División en conjuntos de entrenamiento, validación y prueba

La división en tres subconjuntos del conjunto de datos es de suma importancia, ya que cada uno de ellos juega un rol importante para la creación de un modelo de aprendizaje de máquinas.

* El conjunto de entrenamiento, es por lo regular, el que contiene más muestras, ya que a partir de estos datos el modelo debe aprender y/o generalizar las características lingüísticas de los tweets marcados como xenófobos y no xenófobos de la mejor forma posible. De esta manera, cuando el modelo deba clasificar un texto nuevo, su etiqueta sea correcta.

* El conjunto de validación, es por lo regular, el que contiene menos muestras. Este conjunto nos permitirá realizar múltiples evaluaciones sobre el modelo, ya sea para determinar si existen mejoras al momento de variar cualquier parámetro involucrado con el modelo, aumentar el conjunto de entrenamiento o incluso evaluar la selección de modelos.

* El conjunto de prueba nos permite realizar la evaluación final del modelo. Por lo regular este conjunto solo debe usarse una vez, esto para evitar posibles sesgos.

En el aprendizaje de máquinas es común aplicar el principio de Pareto, el cual sostiene que la división óptima es 80% de los datos para entrenamiento y el restante para tareas de evaluación. Sin embargo, dado que se sabe que el modelo pasará por una serie de algoritmos de optimización, sí se usara esta división podría lograrse un sesgo en cuanto al rendimiento real del modelo. 

Para la división en subconjuntos debe considerarse también la siguiente pregunta ¿El conjunto de datos total está balanceado? Al decir balanceado nos referimos a que las etiquetas disponibles son proporcionales entre sí. Para nuestro caso, dado que se trabaja con únicamente dos etiquetas, diremos que nuestro conjunto de datos es balanceado sí y sólo sí aproximadamente el 50% de los datos contienen la etiqueta *xenófobo*, y el restante corresponde a la etiqueta *no xenofóbo*.

Cuando el conjunto de datos se encuentra desbalanceado (como es el caso) la división en subconjuntos debe hacerse con cuidado, a modo de distribuir proporcionalmente la o las clases más desbalanceadas en cada uno de los tres.

Un ejemplo de omitir esta distribución puede suceder como sigue: Suponga que se tiene un conjunto de datos total de 1000 muestras, donde 780 corrresponden a la clase A y 220 a la clase B. Se hace la división de forma aleatoria a modo de que el conjunto de entrenamiento contiene sólo 20 muestras de la clase B y 680 de la clase A. Tanto el conjunto de validación y prueba contendrán 100 muestras de la clase A y 200 de la clase B. Dado que las muestras de la clase B en el conjunto de entrenamiento son pocas, el modelo no podrá aprender lo suficiente de la clase B y por ende, su rendimiento para esta clase será muy bajo.

In [9]:
#shuffle data before splitting
data = data.sample(frac=1, ignore_index=True)

#vamos a dividir el dataset por su id y etiqueta
X = data.id.values
y = data.label.values
#generate train and test sets
x_train,x_test,y_train,y_test = train_test_split(X, 
                                    y, test_size=0.2, random_state=2022, stratify=y)
#from x_test and y_test, generate validation and test sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.125, random_state=2022,
                                                  stratify=y_train)
                

In [10]:
#get the dataframes
train_df = data[data.id.isin(x_train)].reset_index(drop=True)
test_df = data[data.id.isin(x_test)].reset_index(drop=True)
valid_df = data[data.id.isin(x_val)].reset_index(drop=True)

In [11]:
#check the distribution of the labels by eye
train_df.label.value_counts()

0    5469
1    1531
Name: label, dtype: int64

In [12]:
#check the final datasets
train_df.head(5)

,id,text,date,label
0,1353357604764971008,@usuario @usuario @usuario Claro porque los ve...,2021-10-20,1
1,1143920056621031424,El hashtag alquiler de hashtag viviendas en ha...,2021-11-21,0
2,1220702929478438912,@usuario Que raro.. total si aquí solo entran ...,2021-07-13,1
3,1347148151598190592,@usuario @usuario @usuario Que están en la otr...,2021-09-29,0
4,1169355407757893632,Una persona como Ivanna Trump no representa a ...,2021-11-12,0


In [13]:
#save the datasets
train_df.to_csv('./assets/data/train.csv', index=False)
test_df.to_csv('./assets/data/test.csv', index=False)
valid_df.to_csv('./assets/data/valid.csv', index=False)